<a href="https://colab.research.google.com/github/tapasmahanand/Streamlit_app_ollama_with_Google_colab/blob/main/Streamlit_app_ollama_with_Google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1 - Install Required Libraries:
!pip install streamlit PyMuPDF4LLM pandas openpyxl pyngrok requests  # Added 'requests' for Ollama API calls

# Install Ollama (using a convenient installation script)
!curl -fsSL https://ollama.com/install.sh | sh

# Install and setup ngrok (as in your original code)
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

In [ ]:
# Cell 2 - Import Libraries and Set Up Environment:
import streamlit as st
import fitz  # PyMuPDF4LLM
import pandas as pd
import os
import time
import requests  # For making requests to the Ollama API
from google.colab import files

from google.colab import userdata
# Remove Groq API key setup
# os.environ["GROQ_API_KEY"] = userdata.get('groq_colab_key')
os.environ["NGROK_AUTH_TOKEN"] = userdata.get('NGROK_AUTH_TOKEN')

# Get ngrok auth token from environment
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
!ngrok authtoken {ngrok_token}

# Start Ollama in the background (replace 'model_one' and 'model_two' with your actual model names)
!nohup ollama serve > ollama.log 2>&1 &
!ollama pull gemma2  # 9B Model
!ollama pull llama3.2 # 3B Model
time.sleep(10) # Give Ollama time to start

In [ ]:
!ollama list

NAME               ID              SIZE      MODIFIED       
llama3.2:latest    a80c4f17acd5    2.0 GB    10 seconds ago    
gemma2:latest      ff02c3702f32    5.4 GB    39 seconds ago    


In [ ]:
%%writefile app.py
import streamlit as st
import fitz
import pandas as pd
import os
import time
import requests  # Changed from 'from groq import Groq'
import tempfile
import io

# Ollama API endpoint
OLLAMA_API_URL = "http://localhost:11434/api/generate"

# Define models configuration for Ollama
MODEL_CONFIGS = {
    "model1": {
        "name": "llama3.2",  # Replace with your actual Ollama model name
        "display_name": "llama3.2"
    },
    "model2": {
        "name": "gemma2",  # Replace with your actual Ollama model name
        "display_name": "gemma2"
    }
}

def chunk_text(text, chunk_size=1000):
    """Split text into chunks of approximately chunk_size words."""
    words = text.split()
    chunks = []
    current_chunk = []
    current_size = 0

    for word in words:
        current_chunk.append(word)
        current_size += 1

        if current_size >= chunk_size:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_size = 0

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

def extract_text_from_pdf(pdf_path):
    """Extract text content from PDF file."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        st.error(f"Error processing PDF {pdf_path}: {str(e)}")
        return None

def get_summary_from_ollama(text, model_name, max_words=None):
    """Get summary from Ollama API with specified model and token management."""
    try:
        chunks = chunk_text(text)
        summaries = []

        for i, chunk in enumerate(chunks):
            time.sleep(2) # Rate limiting

            if len(chunks) > 1:
                if i == 0:
                    prompt = f"This is part 1 of {len(chunks)} parts. "
                else:
                    prompt = f"This is part {i+1} of {len(chunks)} parts. "

                if max_words:
                    prompt += f"Please provide a brief summary of this part (the full summary across all parts should be under {max_words} words):\n\n{chunk}"
                else:
                    prompt += f"Please provide a concise summary of this part:\n\n{chunk}"
            else:
                if max_words:
                    prompt = f"Please provide a summary of the following text in under {max_words} words:\n\n{chunk}"
                else:
                    prompt = f"Please provide a concise summary of the following text:\n\n{chunk}"

            payload = {
                "prompt": prompt,
                "model": model_name,
                "stream": False  # Set to False for a single response
            }
            response = requests.post(OLLAMA_API_URL, json=payload)
            response.raise_for_status()  # Raise an exception for bad status codes

            summary = response.json()['response']
            summaries.append(summary)

        if len(summaries) > 1:
            combined_text = " ".join(summaries)
            time.sleep(2) # Rate limiting
            final_prompt = f"Please provide a {'concise' if not max_words else f'under {max_words} words'} summary combining these separate summaries:\n\n{combined_text}"
            payload = {
                "prompt": final_prompt,
                "model": model_name,
                "stream": False
            }
            final_response = requests.post(OLLAMA_API_URL, json=payload)
            final_response.raise_for_status()
            return final_response.json()['response']
        else:
            return summaries[0] if summaries else None

    except requests.exceptions.RequestException as e:
        st.error(f"Error communicating with Ollama API: {e}")
        return None
    except Exception as e:
        st.error(f"Error getting summary from Ollama: {e}")
        return None

def process_pdfs(uploaded_files, selected_model):
    """Process PDFs and generate summaries using the selected Ollama model."""
    results = []

    for uploaded_file in uploaded_files:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
            tmp_file.write(uploaded_file.getvalue())
            tmp_path = tmp_file.name

        text = extract_text_from_pdf(tmp_path)
        os.unlink(tmp_path)  # Clean up temporary file

        if text:
            result = {'PDF_Name': uploaded_file.name}

            # Generate summaries for the selected model
            model_config = next(item for item in MODEL_CONFIGS.values() if item['display_name'] == selected_model)
            model_name = model_config['name']
            display_name = model_config['display_name']

            # Get 50-word summary
            summary_50 = get_summary_from_ollama(text, model_name, max_words=50)
            result[f'{display_name}_50_Words'] = summary_50

            # Get unlimited summary
            summary_unlimited = get_summary_from_ollama(text, model_name)
            result[f'{display_name}_Unlimited'] = summary_unlimited

            results.append(result)

    return results

def export_to_excel(df):
    """Export DataFrame to Excel file in memory."""
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, index=False)
    output.seek(0)
    return output

def main():
    st.title("LLM PDF Summarization")

    # Model selection in sidebar
    st.sidebar.title("Select LLM Model")
    model_options = [config['display_name'] for config in MODEL_CONFIGS.values()]
    selected_model = st.sidebar.selectbox("Choose a model", model_options)

    # File uploader for PDFs
    uploaded_files = st.file_uploader("Upload PDF files", type=['pdf'], accept_multiple_files=True)

    if uploaded_files:
        if st.button("Generate Summaries"):
            with st.spinner(f"Processing PDFs with {selected_model}..."):
                results = process_pdfs(uploaded_files, selected_model)

                # Display results
                for result in results:
                    st.subheader(f"Results for {result['PDF_Name']}")
                    display_name = selected_model
                    st.write(f"{display_name} Summaries:")
                    st.write("50 Words:", result[f'{display_name}_50_Words'])
                    st.write("Unlimited:", result[f'{display_name}_Unlimited'])

                # Create Excel file and provide download button
                df = pd.DataFrame(results)
                excel_file = export_to_excel(df)
                st.download_button(
                    label="Download Excel Report",
                    data=excel_file,
                    file_name="summary_report.xlsx",
                    mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
                )

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
from pyngrok import ngrok
import time

# Kill any existing Streamlit processes
!kill -9 $(pgrep streamlit) 2>/dev/null

# Start Streamlit
!streamlit run app.py &>/content/logs.txt &
time.sleep(5)

# Create ngrok tunnel with correct configuration
ngrok_tunnel = ngrok.connect(addr="8501", proto="http", bind_tls=True)
print(f"Streamlit app URL: {ngrok_tunnel.public_url}")

# Keep the tunnel open
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Closing ngrok tunnel...")
    ngrok.kill()

Streamlit app URL: https://bb81-34-142-192-207.ngrok-free.app
